In [181]:
import requests
import numpy
from lxml import html
from lxml import etree
from io import StringIO, BytesIO
import csv
import pandas as pd
from functools import reduce

In [311]:
def getvalues2(inputstring):
    values = []
    current = 0
    start = 0
    check =  0
    for char in inputstring:
        value = ' '
        if(char == '"' and check == 0):
            start = current
            check = 1
            continue
        if(char == '"' and check == 1):
            value = inputstring[start+len(values)+1:current+len(values)+1]
            values.append(value)
            check = 0
        current = current + 1
    return values

def countmatches(inputstring):
    count = 0
    check = 0
    for char in inputstring:
        if(char == '['):
            check = 1
        if(char == ']') and (check == 1):
            count = count + 1
            check = 0
    return count

def getmatch(startind, page):
    info = []
    nextind = startind
    endind = (page[startind::]).index(']') + startind - 1
    match = page[startind:endind]
    try:
        nextind = (page[endind::1]).index('[') + endind + 1
    except:
        ValueError
    info.append(match)
    info.append(nextind)
    return info

def get_player_match_stats(**kwargs):
    
    #No Spaces and should be capitalized e.g. RafaelNadal
    player_name = kwargs.get('player_name')
    #YEAR-MONTH-DAY format YYYY-MM-DD
    to_date = kwargs.get('to_date')
    to_date = pd.Timestamp(to_date)
    
    #Create url string from playername and base url
    url = 'http://www.tennisabstract.com/cgi-bin/player-classic.cgi?p=%s&f=ACareerqq'%player_name
    #Get response
    response = requests.get(url)
    
    #Get static player stats - constant throughout career so don't need to be recalculated each match
    page_text = response.text
    
    fullname = page_text[page_text.index('var fullname = ')+ len('var fullname = '):page_text.index('var fullname')+page_text[page_text.index('var fullname'):].index(';')]
    dob = page_text[page_text.index('var dob = ')+ len('var dob = '):page_text.index('var dob')+page_text[page_text.index('var dob'):].index(';')]
    height = page_text[page_text.index('var ht = ')+ len('var ht = '):page_text.index('var ht')+page_text[page_text.index('var ht'):].index(';')]
    hand = page_text[page_text.index('var hand = ')+ len('var hand = '):page_text.index('var hand')+page_text[page_text.index('var hand'):].index(';')]
    backhand = page_text[page_text.index('var backhand = ')+ len('var backhand = '):page_text.index('var backhand')+page_text[page_text.index('var backhand'):].index(';')]
    country = page_text[page_text.index('var country = ')+ len('var country = '):page_text.index('var country')+page_text[page_text.index('var country'):].index(';')]

    #Get list of matches as string and then split into individual matches
    startindex = page_text.index('matchmx = [') + len('matchmx = [')
    end_index = page_text[startindex::].index('];')
    matches_string = page_text[startindex:startindex+end_index]
    nomatches = countmatches(matches_string)
    matches = []

    #Loop through string of matches and get individual match info and append to list
    for i in range(nomatches):
        
    #Get each match and create dictionary of stats for that match
        results = getmatch(startindex, page_text)
        match = getvalues2(results[0])

        date = match[0]
        tournament = match[1]
        surface = match[2]
        result = match[4]
        curr_rank = match[5]
        curr_seed = match[6]
        stage = match[8]
        score = match[9]
        no_sets = match[10]
        oppname = match[11]
        opprank = match[12]
        oppseed = match[13]
        opp_hand = match[15]
        opp_elo = match[16]
        opp_height = match[17]
        opp_country = match[18]
        match_time = match[20]
        aces = match[21]
        dfs = match[22]
        sp = match[23]
        sp1 = match[24]
        sp1w = match[25]
        sp2w = match[26]
        bptsaved = match[28]
        bptfaced = match[29]
        opp_aces = match[30]
        opp_dfs = match[31]
        opp_sp = match[32]
        opp1sp = match[33]
        opp1spw = match[34]
        opp2spw = match[35]
        oppbptsaved = match[37]
        oppbpt = match[38]
        opp_backhand = match[39]

        cleanmatch = {'date':pd.Timestamp(date)
                      ,'tournament':tournament
                      ,'surface':surface
                      ,'result':result
                      ,'stage':stage
                      ,'score':score 
                      ,'no_sets':no_sets
                      ,'fullname':fullname
                      ,'curr_rank':curr_rank
                      ,'curr_seed':curr_seed
                      ,'hand':hand
                      ,'backhand':backhand
                      ,'height':height
                      ,'country':country
                      ,'opp_elo':opp_elo
                      ,'oppname':oppname
                      ,'opprank':opprank
                      ,'oppseed':oppseed
                      ,'opp_hand':opp_hand
                      ,'opp_backhand':opp_backhand
                      ,'opp_height':opp_height
                      ,'opp_country':opp_country
                      ,'match_time':match_time
                      ,'aces':aces
                      ,'dfs':dfs
                      ,'sp':sp
                      ,'sp1':sp1
                      ,'sp1w':sp1w
                      ,'sp2w':sp2w
                      ,'bptsaved':bptsaved
                      ,'bptfaced':bptfaced
                      ,'opp_aces':opp_aces
                      ,'opp_dfs':opp_dfs
                      ,'opp_sp':opp_sp
                      ,'opp1sp':opp1sp
                      ,'opp1spw':opp1spw
                      ,'opp2spw':opp2spw
                      ,'oppbptsaved':oppbptsaved
                      ,'oppbpt':oppbpt}
        
    #Get start index of next match for next iteration
        nextindex = results[1]
        startindex = nextindex
        matches.append(cleanmatch)
        
    #Convert to Dataframe and return
    matches_df = pd.DataFrame(matches)
    
    #Filter and return matches before the given date
    matches_df_filtered = matches_df.loc[matches_df['date'] < to_date,:]
    
    return(matches_df_filtered)


In [312]:
nadal_matches_df = get_player_match_stats(player_name='RafaelNadal',to_date='2020-02-25')
nadal_matches_df.head()

,date,tournament,surface,result,stage,score,no_sets,fullname,curr_rank,curr_seed,...,bptsaved,bptfaced,opp_aces,opp_dfs,opp_sp,opp1sp,opp1spw,opp2spw,oppbptsaved,oppbpt
0,2020-02-24,Acapulco,Hard,W,F,6-3 6-2,3,'Rafael Nadal',2,1,...,1,1,10,0,47,30,21,6,4,7
1,2020-02-24,Acapulco,Hard,W,SF,6-3 6-2,3,'Rafael Nadal',2,1,...,4,6,2,1,46,33,16,5,1,6
2,2020-02-24,Acapulco,Hard,W,QF,6-2 6-1,3,'Rafael Nadal',2,1,...,8,8,0,3,47,25,15,8,2,6
3,2020-02-24,Acapulco,Hard,W,R16,6-2 7-5,3,'Rafael Nadal',2,1,...,1,3,3,2,52,28,15,11,1,6
4,2020-02-24,Acapulco,Hard,W,R32,6-3 6-2,3,'Rafael Nadal',2,1,...,2,4,0,3,52,29,12,12,5,10


In [187]:
def split_name_country(name):
    
    player = name[:name.index('(')].strip()
    country = name[name.index('(')+1:name.index(')')].strip()
    
    player_split = player.split(' ')
    first_name = player_split[len(player_split)-1]
    last_name_list = player_split[0:len(player_split)-1]
    last_name = reduce(lambda a,b: a+' '+b, last_name_list)

    return(first_name,last_name,country)

def clean_rank(rank):
    
    clean_rank = rank[rank.index('(')+1:rank.index(')')].strip()
    
    return(clean_rank)

def get_matches_by_date(**kwargs):
    
    year= kwargs.get("year")
    month= kwargs.get("month")
    day= kwargs.get("day")
    
    #Create url string from date and base url
    url = "http://www.tennisprediction.com/?t_p=1&year=%s&month=%s&day=%s"%(year,month,day)
    #Get response
    response = requests.get(url)
    tree = html.fromstring(response.content)
    
    #Get player names, odds, win percentge? and final result
    matches_names = tree.xpath('//table[@id="main_tur"][1]//tr[@class="match"]//td[@class="main_player"]//text() | //table[@id="main_tur"][1]//tr[@class="match1"]//td[@class="main_player"]//text()')
    matches_names.remove('ret.')
    matches_odds = tree.xpath('//table[@id="main_tur"][1]//tr[@class="match"]//td[@class="main_odds_m"]//text() | //table[@id="main_tur"][1]//tr[@class="match1"]//td[@class="main_odds_m"]//text()')
    matches_perc = tree.xpath('//table[@id="main_tur"][1]//tr[@class="match"]//td[@class="main_perc"]//text() | //table[@id="main_tur"][1]//tr[@class="match1"]//td[@class="main_perc"]//text()')
    matches_result = tree.xpath('//table[@id="main_tur"][1]//tr[@class="match"]//td[@class="main_res_f"]//text() | //table[@id="main_tur"][1]//tr[@class="match1"]//td[@class="main_res_f"]//text()')
    matches_sets = tree.xpath('//table[@id="main_tur"][1]//tr[@class="match"]//td[@class="main_res"]//text() | //table[@id="main_tur"][1]//tr[@class="match1"]//td[@class="main_res"]//text()')
    
    match_name_list = []
    match_info_list = []

    for i in range(len(matches_names)):

        m = matches_names[i]
        if i%5 == 0:
            first_name,last_name,country = split_name_country(m)
        elif i%5 == 2:
            rank = clean_rank(m)
        elif i%5 == 3:
            opp_first_name,opp_last_name,opp_country = split_name_country(m)
        elif i%5 == 4:
            opp_rank = clean_rank(m)

            #Create dict of match info

            match_info = {'first_name':first_name,'last_name':last_name,'rank':rank,'country':country,'opp_first_name':opp_first_name,'opp_last_name':opp_last_name,'opp_rank':opp_rank,'opp_country':opp_country}
            match_name_list.append(match_info)

    for i in range(len(matches_result)):

        if i%2 == 0:
            p1_result = matches_result[i]
            p1_odds = matches_perc[i]
            p1_perc = matches_odds[i]
        elif i%2 == 1:
            p2_result = matches_result[i]
            p2_odds = matches_perc[i]
            p2_perc = matches_odds[i]

            #Create dict of match info

            match_info = {'final_score':p1_result,'odds':p1_odds,'perc':p1_perc,'opp_final_score':p2_result,'opp_odds':p2_odds,'opp_perc':p2_perc}
            match_info_list.append(match_info)
            
    match_names_df = pd.DataFrame(match_name_list)
    match_info_df = pd.DataFrame(match_info_list)
    match_info_full_df = match_names_df.join(match_info_df)
    
    return(match_info_full_df)

In [189]:
matches_df = get_matches_by_date(year = 2020,month = 1,day = 30)
matches_df.head()

,first_name,last_name,rank,country,opp_first_name,opp_last_name,opp_rank,opp_country,final_score,odds,perc,opp_final_score,opp_odds,opp_perc
0,Novak,Djokovic,2,SRB,Roger,Federer,3,CHE,3,41.01%,1.11,0,58.99%,6.44
1,Mitchell,Krueger,164,USA,Donald,Young,257,USA,2,60.26%,1.79,1,39.74%,1.92
2,Christopher,Eubanks,221,USA,Michael,Redlicki,377,USA,2,62.07%,1.4,1,37.93%,2.68
3,Steve,Johnson,75,USA,Aleksandar,Vukic,272,AUS,2,55.36%,1.15,1,44.64%,4.6
4,Denis,Istomin,170,UZB,Frances,Tiafoe,50,USA,2,36.49%,2.68,0,63.51%,1.4


In [221]:
def make_name_for_search(**kwargs):
    
    first_name = kwargs.get('first_name')
    last_name = kwargs.get('last_name')
    search_name = first_name + last_name
    
    return(search_name)

def make_name_for_search_bad(**kwargs):
    
    first_name = kwargs.get('first_name')
    last_name = kwargs.get('last_name')
    
    last_name_list = last_name.split(' ')
    new_first_name = last_name_list[len(last_name_list)-1]
    new_last_name = first_name + reduce(lambda a,b: a+b,last_name_list[:len(last_name_list)-1])

    search_name = new_first_name + new_last_name
    
    return(search_name)

In [314]:
search_name = make_name_for_search_bad(first_name=matches_df['first_name'][8]
                                      ,last_name=matches_df['last_name'][8])
to_date = pd.Timestamp(year = 2020,month = 1,day = 30)

player_df = get_player_match_stats(player_name=search_name,to_date=to_date)
player_df.head()

,date,tournament,surface,result,stage,score,no_sets,fullname,curr_rank,curr_seed,...,bptsaved,bptfaced,opp_aces,opp_dfs,opp_sp,opp1sp,opp1spw,opp2spw,oppbptsaved,oppbpt
7,2020-01-27,Newport Beach CH,Hard,L,F,6-4 6-1,3,'Daniel Elahi Galan Riveros',186,,...,3,7,2,0,41,21,17,16,2,2
8,2020-01-27,Newport Beach CH,Hard,W,SF,6-2 7-6(3),3,'Daniel Elahi Galan Riveros',186,,...,1,3,1,8,80,44,27,14,4,8
9,2020-01-27,Newport Beach CH,Hard,W,QF,4-6 7-6(5) 6-3,3,'Daniel Elahi Galan Riveros',186,,...,11,13,6,4,106,74,48,16,6,9
10,2020-01-27,Newport Beach CH,Hard,W,R16,6-3 6-7(6) 6-3,3,'Daniel Elahi Galan Riveros',186,,...,5,8,4,6,91,46,29,24,2,7
11,2020-01-27,Newport Beach CH,Hard,W,R32,6-3 6-1,3,'Daniel Elahi Galan Riveros',186,,...,0,1,0,2,60,33,16,11,4,9


In [332]:
def convert_raw_to_modelling_df(**kwargs):
    
    player_df = kwargs.get('player_df')

    #Clean the data for modelling - remove rows without statistics - usually lower tier tournaments (ITF Futures)
    player_df_clean = player_df.loc[player_df['sp1'] != '',:]

    #First serve percentage
    player_df_clean['sp1_perc'] = player_df_clean['sp1'].astype('double')/player_df_clean['sp'].astype('double')
    #First serve win percentage
    player_df_clean['sp1w_perc'] = player_df_clean['sp1w'].astype('double')/player_df_clean['sp1'].astype('double')
    #Second serve win percentage
    player_df_clean['sp2w_perc'] = player_df_clean['sp2w'].astype('double')/(player_df_clean['sp'].astype('double')-player_df_clean['sp1'].astype('double'))
    #Break point faced percentage
    player_df_clean['bpt_perc'] = player_df_clean['bptfaced'].astype('double')/player_df_clean['sp'].astype('double')
    #Break point save percentage
    player_df_clean['bpt_perc'] = player_df_clean['bptsaved'].astype('double')/player_df_clean['bptfaced'].astype('double')
    #Ace percentage 
    player_df_clean['ace_perc'] = player_df_clean['aces'].astype('double')/player_df_clean['sp'].astype('double')
    #Df percentage
    player_df_clean['df_perc'] = player_df_clean['dfs'].astype('double')/player_df_clean['sp'].astype('double')

    #Opponent first serve percentage
    player_df_clean['opp_sp1_perc'] = player_df_clean['opp1sp'].astype('double')/player_df_clean['opp_sp'].astype('double')
    #Opponent first serve win percentage
    player_df_clean['opp_sp1w_perc'] = player_df_clean['opp1spw'].astype('double')/player_df_clean['opp1sp'].astype('double')
    #Opponent second serve win percentage
    player_df_clean['opp_sp2w_perc'] = player_df_clean['opp2spw'].astype('double')/(player_df_clean['opp_sp'].astype('double')-player_df_clean['opp1sp'].astype('double'))
    #Opponent break point faced percentage
    player_df_clean['opp_bpt_perc'] = player_df_clean['oppbpt'].astype('double')/player_df_clean['opp_sp'].astype('double')
    #Opponent break point saved percentage
    player_df_clean['opp_bpt_perc'] = player_df_clean['oppbptsaved'].astype('double')/player_df_clean['oppbpt'].astype('double')
    #Opponent ace percentage 
    player_df_clean['opp_ace_perc'] = player_df_clean['opp_aces'].astype('double')/player_df_clean['opp_sp'].astype('double')
    #Opponent df percentage
    player_df_clean['opp_df_perc'] = player_df_clean['opp_dfs'].astype('double')/player_df_clean['opp_sp'].astype('double')

    #Reverse order and get rolling means
    player_df_clean_3_match_agg = player_df_clean.loc[:,'sp1_perc':'opp_df_perc'].iloc[::-1].rolling(3).mean()
    player_df_clean_10_match_agg = player_df_clean.loc[:,'sp1_perc':'opp_df_perc'].iloc[::-1].rolling(10).mean()
    #Shift index by 1 to get data from previous games only!
    player_df_clean_3_match_agg = player_df_clean_3_match_agg.shift(1).iloc[::-1]
    player_df_clean_10_match_agg = player_df_clean.loc[:,'sp1_perc':'opp_df_perc'].iloc[::-1].rolling(10).mean()

    #Get relevant columns for predicting match outcome
    player_df_modelling = player_df_clean.loc[:,['result','curr_rank','hand','backhand','opprank','opp_elo','opp_hand','opp_backhand']]
    #Add 3 and 10 match rolling averages of stats
    player_df_modelling = player_df_modelling.join(player_df_clean_3_match_agg,rsuffix='_3match_rolling')
    player_df_modelling = player_df_modelling.join(player_df_clean_10_match_agg,rsuffix='_10match_rolling')
    
    return(player_df_modelling)

In [349]:
def get_player_modelling_df(**kwargs):
    
    player_name = kwargs.get('player_name')
    to_date = kwargs.get('to_date')
    
    player_df = get_player_match_stats(player_name=player_name,to_date=to_date)
    player_df_modelling = convert_raw_to_modelling_df(player_df=player_df)
    
    return(player_df_modelling)

In [335]:
player_df_modelling = get_player_modelling_df(player_name=search_name,to_date=to_date)
player_df_modelling.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,result,curr_rank,hand,backhand,opprank,opp_elo,opp_hand,opp_backhand,sp1_perc,sp1w_perc,...,sp2w_perc_10match_rolling,bpt_perc_10match_rolling,ace_perc_10match_rolling,df_perc_10match_rolling,opp_sp1_perc_10match_rolling,opp_sp1w_perc_10match_rolling,opp_sp2w_perc_10match_rolling,opp_bpt_perc_10match_rolling,opp_ace_perc_10match_rolling,opp_df_perc_10match_rolling
7,L,186,'U','',271,24.9336071184,U,,0.629762,0.680616,...,0.495378,0.612851,0.117846,0.021080,0.610419,0.637105,0.498532,0.561905,0.044035,0.040110
8,W,186,'U','',164,26.0205338809,R,2,0.595503,0.727360,...,0.503653,0.639994,0.118279,0.018482,0.626591,0.630346,0.468532,0.495238,0.050027,0.046632
9,W,186,'U','',221,23.7097878166,R,1,0.631945,0.747423,...,0.505806,0.673328,0.111976,0.019551,0.643170,0.630747,0.470383,0.470238,0.051934,0.037684
10,W,186,'U','',54,20.3887748118,R,,0.647157,0.779249,...,0.493729,0.645855,0.113669,0.018880,0.637359,0.615882,0.487050,0.458117,0.047607,0.036577
11,W,186,'U','',165,19.2334017796,R,2,0.687962,0.773421,...,0.482510,0.652586,0.110018,0.018563,0.654734,0.630616,0.486658,0.504545,0.056419,0.029984


In [336]:
#Create list of player page links to loop through

#Get response from url and convert to xml tree to parse
url = "http://tennisabstract.com/reports/atp_elo_ratings.html"
response = requests.get(url)
tree = html.fromstring(response.content)

#Get player names using xpath to get player page links
player_names = tree.xpath('//td[@align="left"]//a/@href')
player_names = [p.replace('http://www.tennisabstract.com/cgi-bin/player.cgi?p=','') for p in player_names[1:]]
player_names[0:5]

['NovakDjokovic',
 'RafaelNadal',
 'RogerFederer',
 'StefanosTsitsipas',
 'DominicThiem']

In [352]:
full_player_df_list = []
to_date = '2020-06-01'

for p in player_names:
    
    player_df_modelling = get_player_match_stats(player_name=p,to_date=to_date)
    full_player_df_list.append(player_df_modelling)

ValueError: substring not found

In [348]:
full_player_df_list[0]

,result,curr_rank,hand,backhand,opprank,opp_elo,opp_hand,opp_backhand,sp1_perc,sp1w_perc,...,sp2w_perc_10match_rolling,bpt_perc_10match_rolling,ace_perc_10match_rolling,df_perc_10match_rolling,opp_sp1_perc_10match_rolling,opp_sp1w_perc_10match_rolling,opp_sp2w_perc_10match_rolling,opp_bpt_perc_10match_rolling,opp_ace_perc_10match_rolling,opp_df_perc_10match_rolling
0,W,148,'U','',118,32.7994524298,R,1,0.612503,0.790525,...,0.469317,0.491185,0.094188,0.036482,0.594717,0.698473,0.530018,0.707143,0.056757,0.040727
1,W,148,'U','',62,26.546201232,R,2,0.631408,0.740020,...,0.463394,0.520351,0.094982,0.034895,0.594246,0.697516,0.554185,0.707143,0.061153,0.037116
2,L,148,'U','',66,22.7734428474,R,2,0.638418,0.715824,...,0.439505,0.420351,0.095482,0.037395,0.593313,0.670243,0.548772,0.718254,0.061153,0.035365
3,W,148,'U','',990,24.5393566051,U,,0.668997,0.678644,...,0.456929,0.480351,0.114660,0.028595,0.592844,0.656861,0.537767,0.693254,0.062294,0.039325
4,L,146,'U','',70,22.7542778919,R,2,0.644031,0.680538,...,0.464943,0.494897,0.098535,0.025998,0.596369,0.668431,0.561142,0.702778,0.062374,0.037082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,L,1225,'U','',271,26.5215605749,R,,0.516570,0.737064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,L,1029,'U','',334,31.4934976044,L,2,0.502517,0.668552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,W,1029,'U','',1048,23.8028747433,L,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,L,1025,'U','',225,22.5681040383,R,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
